# **Customer Financial Profile Analysis**

## **Background:**
We are working for a financial services company that wants to understand its customers' credit
card usage and overall financial health.

In [27]:
!pip install geopy



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [28]:
!pip install openpyxl


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [29]:
import pandas as pd
import numpy as np
import openpyxl
import matplotlib.pyplot as plt
import seaborn as sns
import hvplot.pandas

%matplotlib inline
sns.set_style("whitegrid")
plt.style.use("fivethirtyeight")

# Load the Cards Dataset

**cards_data**: Contains details about each credit card associated with a user.

In [30]:
df_cards =  pd.read_excel('Pictures/cards_data.xlsx')

In [31]:
df_cards.head()

,card_id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed
0,4524,825,Visa,Debit,4344676511950440,2022-12-01,623,YES,2,24295,2002-09-01,2008
1,2731,825,Visa,Debit,4956965974959980,2020-12-01,393,YES,2,21968,2014-04-01,2014
2,3701,825,Visa,Debit,4582313478255490,2024-02-01,719,YES,2,46414,2003-07-01,2004
3,42,825,Visa,Credit,4879494103069050,2024-08-01,693,NO,1,12400,2003-01-01,2012
4,4659,825,Mastercard,Debit (Prepaid),5722874738736010,2009-03-01,75,YES,1,28,2008-09-01,2009


# Load the Users Dataset

**users_data**: This contains demographic and financial information about the users.

In [32]:
df_users = pd.read_excel("Pictures/users_data.xlsx")
df_users.head()

,main_id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,29278,59696,127613,787,5
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,37891,77254,191349,701,5
2,1718,81,67,1938,11,Female,766 Third Drive,34.02,-117.89,22681,33483,196,698,5
3,708,63,63,1957,1,Female,3 Madison Street,40.71,-73.99,163145,249925,202328,722,4
4,1164,43,70,1976,9,Male,9620 Valley Stream Drive,37.76,-122.44,53797,109687,183855,675,1


## **Merge the two datasets**

Since **_client_id_** in **_cards_data_** corresponds to **_main_id_** in **_users_data_**, we can merge them since the **_client_id_** in **_cards_data_** is a **foreigkey** in the **_users_data_**

### Explanation on the merge() function

We are performing a **left join** between two DataFrames (df_users and df_cards) using pandas' **merge()** function.

In [33]:
pd.set_option('display.max_columns', None)

In [34]:
df = df_users.merge(df_cards, left_on = "main_id", right_on= "client_id", how = "left")

In [35]:
df.head()

,main_id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,card_id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed
0,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,29278,59696,127613,787,5,4524,825,Visa,Debit,4344676511950440,2022-12-01,623,YES,2,24295,2002-09-01,2008
1,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,29278,59696,127613,787,5,2731,825,Visa,Debit,4956965974959980,2020-12-01,393,YES,2,21968,2014-04-01,2014
2,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,29278,59696,127613,787,5,3701,825,Visa,Debit,4582313478255490,2024-02-01,719,YES,2,46414,2003-07-01,2004
3,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,29278,59696,127613,787,5,42,825,Visa,Credit,4879494103069050,2024-08-01,693,NO,1,12400,2003-01-01,2012
4,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,29278,59696,127613,787,5,4659,825,Mastercard,Debit (Prepaid),5722874738736010,2009-03-01,75,YES,1,28,2008-09-01,2009


## **Converting Latitude and Longitude into City and State**

For a faster solution, you can use the **reverse_geocoder package**, which performs offline reverse geocoding using a prebuilt database of places. This method is typically much faster than making online API calls with geopy.

In [36]:
!pip install reverse_geocoder


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [37]:
import pandas as pd
import reverse_geocoder as rg

# Convert latitude and longitude to numeric values, coercing errors to NaN
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')

# Drop rows with invalid coordinates
df = df.dropna(subset=['latitude', 'longitude'])

# Convert each row's latitude and longitude into a tuple.
coordinates = list(df[['latitude', 'longitude']].apply(lambda row: (row['latitude'], row['longitude']), axis=1))

# Use reverse_geocoder in batch mode
results = rg.search(coordinates)

# Extract the city (key 'name') and state (key 'admin1') from each result
df['city'] = [res['name'] for res in results]
df['state'] = [res['admin1'] for res in results]

# Display a sample of the updated DataFrame
print(df[['latitude', 'longitude', 'city', 'state']].head(100))


    latitude  longitude                 city        state
0      34.15    -117.76             La Verne   California
1      34.15    -117.76             La Verne   California
2      34.15    -117.76             La Verne   California
3      34.15    -117.76             La Verne   California
4      34.15    -117.76             La Verne   California
..       ...        ...                  ...          ...
95     41.29     -72.36  Old Saybrook Center  Connecticut
96     30.68     -88.04               Mobile      Alabama
97     30.68     -88.04               Mobile      Alabama
98     30.68     -88.04               Mobile      Alabama
99     38.48    -121.34             Vineyard   California

[100 rows x 4 columns]


In [38]:
df[['latitude', 'longitude']]

,latitude,longitude
0,34.15,-117.76
1,34.15,-117.76
2,34.15,-117.76
3,34.15,-117.76
4,34.15,-117.76
...,...,...
6141,40.66,-74.19
6142,40.66,-74.19
6143,40.24,-76.92
6144,42.86,-71.48


In [39]:
df.describe()

,main_id,current_age,retirement_age,birth_year,birth_month,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,card_id,client_id,card_number,expires,cvv,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed
count,6007.000000,6007.000000,6007.000000,6007.000000,6007.000000,6007.000000,6007.000000,6007.000000,6007.000000,6007.000000,6007.000000,6007.000000,6007.000000,6007.000000,6.007000e+03,6007,6007.000000,6007.000000,6007.000000,6007,6007.000000
mean,996.419011,50.066423,66.525054,1969.123689,6.457966,37.527586,-91.744245,23284.618945,45379.759281,57400.388713,718.249209,3.938905,3077.327118,996.419011,4.816336e+15,2020-10-09 02:54:02.217413120,506.606626,1.501415,14357.801065,2011-01-15 02:16:52.818378496,2013.436990
min,0.000000,18.000000,50.000000,1918.000000,1.000000,20.880000,-159.410000,0.000000,1.000000,0.000000,480.000000,1.000000,0.000000,0.000000,3.001055e+14,1997-07-01 00:00:00,0.000000,1.000000,0.000000,1991-01-01 00:00:00,2002.000000
25%,487.000000,35.000000,65.000000,1957.000000,3.000000,33.940000,-97.510000,16890.000000,32330.000000,16764.000000,687.000000,3.000000,1542.500000,487.000000,4.480625e+15,2020-02-01 00:00:00,257.000000,1.000000,7022.000000,2006-10-01 00:00:00,2010.000000
50%,998.000000,50.000000,66.000000,1970.000000,7.000000,38.410000,-86.830000,20670.000000,40334.000000,51026.000000,717.000000,4.000000,3077.000000,998.000000,5.107151e+15,2021-09-01 00:00:00,517.000000,1.000000,12596.000000,2010-02-01 00:00:00,2013.000000
75%,1503.000000,63.000000,69.000000,1984.000000,10.000000,41.200000,-80.080000,26369.000000,52630.000000,83995.000000,760.000000,5.000000,4617.500000,1503.000000,5.584137e+15,2023-05-01 00:00:00,757.500000,2.000000,19164.500000,2016-05-01 00:00:00,2017.000000
max,1999.000000,101.000000,79.000000,2002.000000,12.000000,61.200000,-68.670000,163145.000000,307018.000000,516263.000000,850.000000,9.000000,6145.000000,1999.000000,6.997197e+15,2024-12-01 00:00:00,999.000000,3.000000,151223.000000,2020-02-01 00:00:00,2020.000000
std,581.654443,18.901996,3.462947,18.898678,3.550247,4.995745,16.280747,11576.642350,23440.823587,51557.017909,63.179733,1.660524,1774.662466,581.654443,1.333335e+15,NaN,289.545503,0.519314,12050.887477,NaN,4.261591


In [40]:
df.dtypes

main_id                           int64
current_age                       int64
retirement_age                    int64
birth_year                        int64
birth_month                       int64
gender                           object
address                          object
latitude                        float64
longitude                       float64
per_capita_income                 int64
yearly_income                     int64
total_debt                        int64
credit_score                      int64
num_credit_cards                  int64
card_id                           int64
client_id                         int64
card_brand                       object
card_type                        object
card_number                       int64
expires                  datetime64[ns]
cvv                               int64
has_chip                         object
num_cards_issued                  int64
credit_limit                      int64
acct_open_date           datetime64[ns]


In [64]:
df = df.drop(columns=['latitude', 'longitude'])

In [65]:
df

,main_id,current_age,retirement_age,birth_year,birth_month,gender,address,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,card_id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,city,state,years_until_retirement,debt_to_income_ratio
0,825,53,66,1966,11,Female,462 Rose Lane,29278,59696,127613,787,5,4524,825,Visa,Debit,4344676511950440,2022-12-01,623,YES,2,24295,2002-09-01,2008,La Verne,California,13,2.137714
1,825,53,66,1966,11,Female,462 Rose Lane,29278,59696,127613,787,5,2731,825,Visa,Debit,4956965974959980,2020-12-01,393,YES,2,21968,2014-04-01,2014,La Verne,California,13,2.137714
2,825,53,66,1966,11,Female,462 Rose Lane,29278,59696,127613,787,5,3701,825,Visa,Debit,4582313478255490,2024-02-01,719,YES,2,46414,2003-07-01,2004,La Verne,California,13,2.137714
3,825,53,66,1966,11,Female,462 Rose Lane,29278,59696,127613,787,5,42,825,Visa,Credit,4879494103069050,2024-08-01,693,NO,1,12400,2003-01-01,2012,La Verne,California,13,2.137714
4,825,53,66,1966,11,Female,462 Rose Lane,29278,59696,127613,787,5,4659,825,Mastercard,Debit (Prepaid),5722874738736010,2009-03-01,75,YES,1,28,2008-09-01,2009,La Verne,California,13,2.137714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6141,185,47,67,1973,1,Female,276 Fifth Boulevard,15175,30942,71066,779,3,5361,185,Amex,Credit,300609782832003,2024-01-01,663,YES,1,6900,2000-11-01,2013,Elizabeth,New Jersey,20,2.296749
6142,185,47,67,1973,1,Female,276 Fifth Boulevard,15175,30942,71066,779,3,2711,185,Visa,Credit,4718517475996010,2021-01-01,492,YES,2,5700,2012-04-01,2012,Elizabeth,New Jersey,20,2.296749
6143,1007,66,60,1954,2,Male,259 Valley Boulevard,25336,54654,27241,618,1,1305,1007,Mastercard,Credit,5929512204765910,2020-08-01,237,NO,2,9200,2012-02-01,2012,Camp Hill,Pennsylvania,-6,0.498426
6144,1110,21,60,1998,11,Female,472 Ocean View Street,32325,65909,181261,673,2,743,1110,Mastercard,Debit,5589768928167460,2020-01-01,630,YES,1,28074,2020-01-01,2020,East Merrimack,New Hampshire,39,2.750171


# 1. **Customer Demographics**

## + What is the age distribution ?

In [42]:
hist_plot = df['current_age'].hvplot.hist(
    bins=20,                      # Number of bins
    color='blue',                 # Color of the histogram
    alpha=0.6,                    # Transparency
    title="Distribution of Current Age",  # Plot title
    xlabel="Age",                 # X-axis label
    ylabel="Frequency",           # Y-axis label
    width=600,                    # Plot width
    height=400                    # Plot height
)

# Display the plot
hist_plot

:Histogram   [current_age]   (Count)

In [43]:
hist_plot = df['retirement_age'].hvplot.hist(
    bins=20,                      # Number of bins
    color='red',                  # Color of the histogram
    alpha=0.6,                    # Transparency
    title="Distribution of Retirement Age",  # Plot title
    xlabel="Age",                 # X-axis label
    ylabel="Frequency",           # Y-axis label
    width=600,                    # Plot width
    height=400                    # Plot height
)

# Display the plot
hist_plot

:Histogram   [retirement_age]   (Count)

## Calculate the Time Until Retirement

In [44]:
df['years_until_retirement'] = df['retirement_age'] - df['current_age']

In [45]:
df['years_until_retirement'].describe()

count    6007.000000
mean       16.458632
std        19.185051
min       -38.000000
25%         3.000000
50%        17.000000
75%        31.000000
max        57.000000
Name: years_until_retirement, dtype: float64

## Visualize Remaining Years Until Retirement

In [46]:
hist_plot = df['years_until_retirement'].hvplot.hist(
    bins=20,                      # Number of bins
    kde=True,                     # Add Kernel Density Estimate (KDE)
    color='purple',               # Color of the histogram
    title="Years Until Retirement Distribution",  # Plot title
    xlabel="Years Until Retirement",              # X-axis label
    ylabel="Frequency",                           # Y-axis label
    width=600,                    # Plot width
    height=400                    # Plot height
)

# Display the plot
hist_plot

:Histogram   [years_until_retirement]   (Count)

## Insights:
. If most values are high (e.g., 20+ years), the company might focus on long-term financial planning.

. If many customers have less than 5 years, they might be interested in retirement financial products.

## - Gender Ratio:

In [47]:
df['gender'].value_counts().hvplot.bar(
    title="Gender Ratio",
    xlabel="Gender",
    ylabel="Count",
    color=['skyblue'],
    height=400,
    width=600
)



:Bars   [gender]   (count)

## Income Distribution

In [48]:
df.hvplot.box(
    y=['yearly_income', 'per_capita_income'], 
    title="Yearly Income vs Per Capita Income Distribution", 
    height=400, 
    width=600
)

:BoxWhisker   [Variable]   (value)

# 2. Credit Card Ownership:

## 2.1. Average Number of Credit Cards per User:

In [49]:
avg_cards = df['num_credit_cards'].mean()
print(f"Average number of credit cards per user: {avg_cards:.2f}")

Average number of credit cards per user: 3.94


In [50]:
# Create a bar plot for credit card counts
plot = df['num_credit_cards'].value_counts().hvplot.bar(
    title="Credit Card Ownership Distribution",
    xlabel="Number of Credit Cards",
    ylabel="Count of Users",
    width=600,
    height=400
)


In [51]:
plot

:Bars   [num_credit_cards]   (count)

## 2.2. Customers with Multiple Credit Cards:

In [52]:
multi_card_users = df[df['num_credit_cards'] > 1]
multi_card_users[['main_id', 'num_credit_cards']]


,main_id,num_credit_cards
0,825,5
1,825,5
2,825,5
3,825,5
4,825,5
...,...,...
6140,185,3
6141,185,3
6142,185,3
6144,1110,2


In [53]:
multi_card_users.hvplot.hist(
    y='num_credit_cards',
    bins = 20,
    title="Distribution of Credit Cards Among Users",
    xlabel="Number of Credit Cards",
    ylabel="Number of Users",
    width=600,
    height=400
)


:Histogram   [num_credit_cards]   (Count)

## 3. Credit Risk Analysis

## 3.1 customers with a debt-to-income ratio > 0.5:

In [54]:
df['debt_to_income_ratio'] = df['total_debt'] / df['yearly_income']

high_risk_users = df[df['debt_to_income_ratio'] > 0.5]

high_risk_users[['main_id', 'total_debt', 'yearly_income', 'debt_to_income_ratio']]


,main_id,total_debt,yearly_income,debt_to_income_ratio
0,825,127613,59696,2.137714
1,825,127613,59696,2.137714
2,825,127613,59696,2.137714
3,825,127613,59696,2.137714
4,825,127613,59696,2.137714
...,...,...,...,...
6140,185,71066,30942,2.296749
6141,185,71066,30942,2.296749
6142,185,71066,30942,2.296749
6144,1110,181261,65909,2.750171


In [55]:
# Group by 'gender' and count the number of high-risk customers in each group
high_risk_users_by_gender = high_risk_users.groupby('gender').size().reset_index(name='count')

# Display the result
print(high_risk_users_by_gender)

# Total number of high-risk customers
total_high_risk_users = high_risk_users_by_gender['count'].sum()
print(f"Total number of high-risk customers: {total_high_risk_users}")

   gender  count
0  Female   2178
1    Male   2229
Total number of high-risk customers: 4407


In [56]:
# Create a bar plot
bar_plot = high_risk_users_by_gender.hvplot.bar(
    x='gender',
    y='count',
    title='High-Risk Customers by Gender',
    xlabel='Gender',
    ylabel='Number of Customers',
    color='magenta',
    rot=45
)

# Display the plot
bar_plot

:Bars   [gender]   (count)

## 4. Card Expiration Analysis

## 4.1. cards expiring within the next year:

In [57]:
df.dtypes

main_id                            int64
current_age                        int64
retirement_age                     int64
birth_year                         int64
birth_month                        int64
gender                            object
address                           object
latitude                         float64
longitude                        float64
per_capita_income                  int64
yearly_income                      int64
total_debt                         int64
credit_score                       int64
num_credit_cards                   int64
card_id                            int64
client_id                          int64
card_brand                        object
card_type                         object
card_number                        int64
expires                   datetime64[ns]
cvv                                int64
has_chip                          object
num_cards_issued                   int64
credit_limit                       int64
acct_open_date  

In [58]:
from datetime import datetime, timedelta

one_year_from_now = datetime.today() + timedelta(days=365)
df['expires'] = pd.to_datetime(df['expires'])

expiring_soon = df[df['expires'] <= one_year_from_now]
print(f"Number of cards expiring within a year: {len(expiring_soon)}")


Number of cards expiring within a year: 6007


## 5. Insights for Marketing:

## 5.1. Ideal customers for premium services:

In [59]:
# Filter premium targets
premium_targets = df[(df['credit_score'] > 700) & (df['yearly_income'] > 50000)]

# Group by 'Gender' and count the number of customers in each group
premium_targets_by_gender = premium_targets.groupby('gender').size().reset_index(name='count')

# Display the result
print(premium_targets_by_gender)

# Total number of premium targets
total_premium_targets = premium_targets_by_gender['count'].sum()
print(f"Total number of premium target customers: {total_premium_targets}")

   gender  count
0  Female    610
1    Male    539
Total number of premium target customers: 1149


In [60]:
# Create an interactive bar plot
bar_plot = premium_targets_by_gender.hvplot.bar(
    x='gender',                  # X-axis: Gender
    y='count',                   # Y-axis: Count of premium targets
    title='Premium Targets by Gender',  # Plot title
    xlabel='gender',             # X-axis label
    ylabel='Number of Customers', # Y-axis label
                # Color bars by Gender
    rot=45                       # Rotate x-axis labels for better readability
)

# Display the plot
bar_plot

:Bars   [gender]   (count)

## 5.2. Customers who might need financial assistance:

In [61]:
assistance_targets = df[(df['credit_score'] < 600) & (df['total_debt'] > df['yearly_income'] * 0.6)]

# Group by 'gender' and count the number of customers in each group
assistance_targets_by_gender = assistance_targets.groupby('gender').size().reset_index(name='count')

# Display the result
print(assistance_targets_by_gender)

   gender  count
0  Female    103
1    Male    117


In [62]:
# Create a bar plot
bar_plot = assistance_targets_by_gender.hvplot.bar(
    x='gender',
    y='count',
    title='Customers by Gender who need financial assistance',
    xlabel='Gender',
    ylabel='Number of Customers',
    color=['lightgreen','skyblue'],
    rot=45
)

# Display the plot
bar_plot

:Bars   [gender]   (count)

In [66]:
df

,main_id,current_age,retirement_age,birth_year,birth_month,gender,address,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,card_id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,city,state,years_until_retirement,debt_to_income_ratio
0,825,53,66,1966,11,Female,462 Rose Lane,29278,59696,127613,787,5,4524,825,Visa,Debit,4344676511950440,2022-12-01,623,YES,2,24295,2002-09-01,2008,La Verne,California,13,2.137714
1,825,53,66,1966,11,Female,462 Rose Lane,29278,59696,127613,787,5,2731,825,Visa,Debit,4956965974959980,2020-12-01,393,YES,2,21968,2014-04-01,2014,La Verne,California,13,2.137714
2,825,53,66,1966,11,Female,462 Rose Lane,29278,59696,127613,787,5,3701,825,Visa,Debit,4582313478255490,2024-02-01,719,YES,2,46414,2003-07-01,2004,La Verne,California,13,2.137714
3,825,53,66,1966,11,Female,462 Rose Lane,29278,59696,127613,787,5,42,825,Visa,Credit,4879494103069050,2024-08-01,693,NO,1,12400,2003-01-01,2012,La Verne,California,13,2.137714
4,825,53,66,1966,11,Female,462 Rose Lane,29278,59696,127613,787,5,4659,825,Mastercard,Debit (Prepaid),5722874738736010,2009-03-01,75,YES,1,28,2008-09-01,2009,La Verne,California,13,2.137714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6141,185,47,67,1973,1,Female,276 Fifth Boulevard,15175,30942,71066,779,3,5361,185,Amex,Credit,300609782832003,2024-01-01,663,YES,1,6900,2000-11-01,2013,Elizabeth,New Jersey,20,2.296749
6142,185,47,67,1973,1,Female,276 Fifth Boulevard,15175,30942,71066,779,3,2711,185,Visa,Credit,4718517475996010,2021-01-01,492,YES,2,5700,2012-04-01,2012,Elizabeth,New Jersey,20,2.296749
6143,1007,66,60,1954,2,Male,259 Valley Boulevard,25336,54654,27241,618,1,1305,1007,Mastercard,Credit,5929512204765910,2020-08-01,237,NO,2,9200,2012-02-01,2012,Camp Hill,Pennsylvania,-6,0.498426
6144,1110,21,60,1998,11,Female,472 Ocean View Street,32325,65909,181261,673,2,743,1110,Mastercard,Debit,5589768928167460,2020-01-01,630,YES,1,28074,2020-01-01,2020,East Merrimack,New Hampshire,39,2.750171
